Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [12]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get("acc")>DESIRED_ACCURACY):
      print("\nWe get our Desired Accuracy")
      self.model.stop_training=True

callbacks = myCallback()

--2019-08-22 15:36:52--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2019-08-22 15:36:52 (168 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [13]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(16,(3,3),activation="relu",input_shape=(300,300,3)),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(32,(3,3),activation="relu"),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64,(3,3),activation="relu"),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(128,activation="relu"),
tf.keras.layers.Dense(1,activation="sigmoid")
])

model.summary()
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.0001),loss="binary_crossentropy",metrics=["acc"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 78400)            

In [14]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
      "/tmp/h-or-s/",
      target_size = (300,300),
      batch_size = 8,
      class_mode = 'binary'
      
)

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [15]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 10,
    epochs=15,
    verbose=2,
    callbacks=[callbacks]
)
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
10/10 - 8s - loss: 0.6979 - acc: 0.6000
Epoch 2/15
10/10 - 7s - loss: 0.4074 - acc: 0.8625
Epoch 3/15
10/10 - 7s - loss: 0.2396 - acc: 0.9000
Epoch 4/15
10/10 - 7s - loss: 0.2059 - acc: 0.8875
Epoch 5/15
10/10 - 7s - loss: 0.1352 - acc: 0.9375
Epoch 6/15
10/10 - 7s - loss: 0.1423 - acc: 0.9500
Epoch 7/15
10/10 - 7s - loss: 0.1060 - acc: 0.9750
Epoch 8/15
10/10 - 7s - loss: 0.1027 - acc: 0.9500
Epoch 9/15
10/10 - 7s - loss: 0.0769 - acc: 0.9750
Epoch 10/15
10/10 - 7s - loss: 0.0765 - acc: 0.9625
Epoch 11/15
10/10 - 7s - loss: 0.0339 - acc: 0.9875
Epoch 12/15
10/10 - 7s - loss: 0.0695 - acc: 0.9625
Epoch 13/15
10/10 - 7s - loss: 0.0408 - acc: 0.9875
Epoch 14/15

We get our Desired Accuracy
10/10 - 7s - loss: 0.0317 - acc: 1.0000
